In [ ]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent / "src"))

import pandas as pd
import plotly.express as px

In [ ]:
commits = pd.read_csv('commits.csv', index_col=0, parse_dates=['date'])
df = commits.sort_values(by='date').reset_index().drop(columns=['index'])
df['weight'] = df['ins'] + df['del']

In [ ]:
df = df.loc[~df["path"].astype(str).str.endswith("package-lock.json")]
df = df.loc[~df["path"].astype(str).str.endswith("custom.css")]
df = df.loc[~df["path"].astype(str).str.endswith("custom.css.map")]
df = df.loc[~df["path"].astype(str).str.endswith("realm.json")]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew")]
df = df.loc[~df["path"].astype(str).str.endswith("gradlew.bat")]
df = df.loc[~df["path"].astype(str).str.endswith(".png")]
df = df.loc[~df["path"].astype(str).str.endswith(".jar")]
df = df.loc[~df["path"].astype(str).str.startswith("build/")]

In [ ]:
from private import email_dict
df = df.replace({'email': email_dict})

In [ ]:
# sum lines by commit (drop repo/file information)
df1 = df.drop(columns=['ins', 'del', 'dir', 'path']).groupby(['date', 'email']).sum()
# sum previous weight values, by email, for plotting
df1['contrib'] = df1.groupby('email')['weight'].transform(pd.Series.cumsum)
# convert data, email indexes to columns
df2 = df1.reset_index()

In [ ]:
df2['weight'] = df2['weight'].clip(upper=10000)

In [ ]:
fig = px.line(df2, x='date', y='weight', color='email', markers=True)
fig.show()